In [5]:
import pandas as pd
from datetime import datetime

files = ["job_1999101.json","job_1988758.json"]

seen_ids = set()

for file in files:
    file_name = "/dccstor/srosent2/generative/appen/" + file
    data = pd.read_json(file_name, lines=True)

    for index, row in data.iterrows():
        info = row['results']['judgments'][0]
        # print(row['results'])
            
        seen_ids.add(info['unit_data']['question_id'])

print(len(seen_ids))
print(seen_ids)

157
{'6663437261326194572', '7467223419449849336', '-456565248592064904', '-3659701405687069690', '-4583177527895959138', '-6816620259919283888', '7195430905493029565', '5738144377210062200', '5875046726135070179', '-72983375937706663', '-547285585922035520', '-5433421993461538564', '8717587106725834760', '6242654346399864962', '-8083017426536732807', '1936491352997888449', '1319546935861782187', '3550133641087898941', '751853662280595560', '2102201014821122308', '5337089324356998680', '-680447666417234559', '3016414031422933067', '9205580801476043040', '9064547148563681840', '-5827705595584518308', '-2353029178083370761', '-6905127158875357602', '2869997669348576087', '1083726726672316078', '-5957516079482453552', '-6616030944495513115', '-4524305586196594191', '-1813693802046160487', '2077474049144747505', '8484713537476807677', '-7773020239294558774', '-8511427837138505122', '-1930131592806504146', '-3224127094587318853', '409959415093042172', '-8277313852204797994', '24080026277339

In [1]:
# load tydi json and compute stats on the questions and answers
import glob
import gzip
import json
import spacy

nlp = spacy.load('en_core_web_sm')

train_data_dir = "/dccstor/srosent2/primeqa/data/nq-lfqa-train/*jsonl"
dev_data_dir = "/dccstor/srosent2/primeqa/data/dev/nq-lfqa/*jsonl"

train_files = glob.glob(train_data_dir)
dev_files = glob.glob(dev_data_dir)

def load_json_from_file(gt_file_patterns):
    data = []
    if gt_file_patterns.endswith('gz'):
        f = gzip.open(gt_file_patterns, 'rt', encoding='utf-8')
    else:
        f = open(gt_file_patterns, 'rt', encoding='utf-8')
    lines = f.readlines()
    for line in lines:
        data.append(json.loads(line))
    return data

In [2]:
train_data = []
for train_file in train_files:
    train_data.extend(load_json_from_file(train_file))

In [11]:
def get_appen_data(data, type, exclude_ids):
    count = 0
    for example in data:
        qtype = example['type'][0]
        
        if qtype != type:
            continue
        
        example_id = example['example_id']

        if str(example_id) in exclude_ids:
            continue

        question = example["question_text"]

        q_word = question.split()[0]

        annotation = example['annotations'][0]

        if annotation == None:
            continue

        minimal_text = ""
        if annotation['minimal_answer']['plaintext_start_byte'] != -1:
            minimal_text = example['document_plaintext'].encode('utf-8')[annotation['minimal_answer']['plaintext_start_byte']:annotation['minimal_answer']['plaintext_end_byte']].decode('utf-8')
        passage_offsets = example['passage_answer_candidates'][annotation['passage_answer']['candidate_index']]
        passage_text = example['document_plaintext'].encode('utf-8')[passage_offsets['plaintext_start_byte']:passage_offsets['plaintext_end_byte']].decode('utf-8')
        
        passage_sentences = ""
        for sentence in nlp(passage_text).sents:
            passage_sentences += sentence.text + "&nbsp;"

        count += 1
        if count >= 30:
            break

        print(str(example_id) + "\t" + question + "\t" + example['document_title'] + "\t" + example['document_url'] + "\t" + passage_sentences + "\t" + minimal_text)
           

In [12]:
get_appen_data(train_data, "la", seen_ids)

-7210339619807281790	what kind of bird is black and white with red on its chest	Rose-breasted grosbeak	https://en.wikipedia.org//w/index.php?title=Rose-breasted_grosbeak&amp;oldid=839872078	The adult male in breeding plumage has a black head , wings , back , and tail , and a bright rose - red patch on its breast ; the wings have two white patches and rose - red linings .&nbsp;Its underside and rump are white .&nbsp;Males in nonbreeding plumage have largely white underparts , supercilium , and cheeks .&nbsp;The upperside feathers have brown fringes , and most wing feathers white ones , giving a scaly appearance .&nbsp;The bases of the primary remiges are also white .&nbsp;The coloration renders the adult male rose - breasted grosbeak ( even while wintering ) unmistakable if seen well .&nbsp;	
4766418881317162187	who sings the song story of an artist	Daniel Johnston	https://en.wikipedia.org//w/index.php?title=Daniel_Johnston&amp;oldid=853705998	In 2018 , Johnston 's 1982 song `` The Stor